In [1]:
# just here
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import iblphotometry.io as ffio

In [4]:
# ONE instantiation
from one.api import ONE

one = ONE(base_url='https://alyx.internationalbrainlab.org', mode='remote')

lab = 'cortexlab'  # for carolinas dataset or 'wittenlab' for alejandros dataset
eids = one.search(dataset='photometry.signal.pqt', lab=lab)
eid = eids[2]

### loading the raw neurophotometrics output from disk
as it is stored on disk. This directly loads into the final data format, which is a python `dict`, the keys are the individual acquisition channel names, and their corresponding values are `pd.DataFrames` that have the timestamps for the frames as index, and as columns the names of the ROIs as selected in the user interface

In [6]:
# raw_neurophotometris = one.load_dataset(eid, '_neurophotometrics_fpData.raw')
raw_photometry = ffio.from_raw_neurophotometrics_file('raw_photometry.csv')
raw_photometry

{'GCaMP':                   G0        G1        G2        G3        G4
 times                                                       
 227.248224  0.009192  0.009263  0.004893  0.005485  0.014840
 227.281536  0.009219  0.009292  0.004903  0.005487  0.014874
 227.314880  0.009220  0.009299  0.004895  0.005498  0.014895
 227.348224  0.009204  0.009299  0.004899  0.005490  0.014878
 227.381536  0.009216  0.009290  0.004895  0.005486  0.014858
 ...              ...       ...       ...       ...       ...
 260.613664  0.004811  0.004186  0.003971  0.004169  0.004815
 260.647008  0.004809  0.004181  0.003974  0.004171  0.004814
 260.680352  0.004811  0.004188  0.003974  0.004170  0.004816
 260.713664  0.004813  0.004184  0.003972  0.004172  0.004824
 260.747008  0.004817  0.004183  0.003972  0.004172  0.004817
 
 [1006 rows x 5 columns],
 'Isosbestic':                   G0        G1        G2        G3        G4
 times                                                       
 227.264896  0.0126

### loading raw neurophotometrics data via one from alyx
the data is stored in alyx in an intermediate format in the form of `photometry.signal.pqt`. This format is, like the raw neurophotometrics files shown above, still time multiplexed with different acquisition channels recorded in series, but the time vector is now aligned to the task events.|

In [7]:
raw_signal = one.load_dataset(eid, dataset='photometry.signal')
raw_signal

,G0,G4,times,valid,wavelength,name,color
0,0.009152,0.016492,-370.181357,False,NaN,,
1,0.011645,0.019462,-370.164717,False,470.0,GCaMP,Blue
2,0.011773,0.019487,-370.148046,False,415.0,Isosbestic,Violet
3,0.011657,0.019477,-370.131374,False,470.0,GCaMP,Blue
4,0.010041,0.017752,-370.114734,False,415.0,Isosbestic,Violet
...,...,...,...,...,...,...,...
187984,0.012575,0.020662,2762.724377,False,415.0,Isosbestic,Violet
187985,0.005855,0.009810,2762.741048,False,470.0,GCaMP,Blue
187986,0.012590,0.020651,2762.757720,False,415.0,Isosbestic,Violet
187987,0.005837,0.009800,2762.774392,False,470.0,GCaMP,Blue


For analysis purposes, this data representation can be split into the individual acquisition channels into the same format as shown above for reading the raw data, a `dict` of `pd.dataframes`

In [9]:
ffio.from_ibl_dataframe(raw_signal)

{'GCaMP':                     G0        G4
 times                           
 -370.164717   0.011645  0.019462
 -370.131374   0.011657  0.019477
 -370.098063   0.011656  0.019509
 -370.064719   0.011674  0.019500
 -370.031408   0.011659  0.019485
 ...                ...       ...
  2762.641050  0.005847  0.009813
  2762.674394  0.005803  0.009814
  2762.707705  0.005834  0.009803
  2762.741048  0.005855  0.009810
  2762.774392  0.005837  0.009800
 
 [93994 rows x 2 columns],
 'Isosbestic':                     G0        G4
 times                           
 -370.148046   0.011773  0.019487
 -370.114734   0.010041  0.017752
 -370.081391   0.011781  0.019512
 -370.048048   0.011775  0.019507
 -370.014736   0.011785  0.019465
 ...                ...       ...
  2762.657722  0.012615  0.020670
  2762.691033  0.012602  0.020660
  2762.724377  0.012575  0.020662
  2762.757720  0.012590  0.020651
  2762.791032  0.012588  0.020645
 
 [93994 rows x 2 columns]}

so far, the column names of the data are not mapped to the brain regions where the fibers are implanted. When retrieving data from alyx, the locations have been registered

In [11]:
raw_photometry_df = one.load_dataset(eid, dataset='photometry.signal')
locations_df = one.load_dataset(eid, dataset='photometryROI.locations')

ffio.from_ibl_dataframes(raw_photometry_df, locations_df)

{'GCaMP':                     SI       PPT
 times                           
 -370.164717   0.011645  0.019462
 -370.131374   0.011657  0.019477
 -370.098063   0.011656  0.019509
 -370.064719   0.011674  0.019500
 -370.031408   0.011659  0.019485
 ...                ...       ...
  2762.641050  0.005847  0.009813
  2762.674394  0.005803  0.009814
  2762.707705  0.005834  0.009803
  2762.741048  0.005855  0.009810
  2762.774392  0.005837  0.009800
 
 [93994 rows x 2 columns],
 'Isosbestic':                     SI       PPT
 times                           
 -370.148046   0.011773  0.019487
 -370.114734   0.010041  0.017752
 -370.081391   0.011781  0.019512
 -370.048048   0.011775  0.019507
 -370.014736   0.011785  0.019465
 ...                ...       ...
  2762.657722  0.012615  0.020670
  2762.691033  0.012602  0.020660
  2762.724377  0.012575  0.020662
  2762.757720  0.012590  0.020651
  2762.791032  0.012588  0.020645
 
 [93994 rows x 2 columns]}

equivalently, the same can be achieved from stored files from disk:

In [12]:
raw_photometry_df = one.load_dataset(
    eid, dataset='photometry.signal', download_only=True
)
signal_pqt_path = one.eid2path(eid) / 'alf/photometry/photometry.signal.pqt'
locations_df = one.load_dataset(
    eid, dataset='photometryROI.locations', download_only=True
)
locations_pqt_path = one.eid2path(eid) / 'alf/photometry/photometryROI.locations.pqt'

ffio.from_ibl_pqt(signal_pqt_path, locations_pqt_path)

{'GCaMP':                     SI       PPT
 times                           
 -370.164717   0.011645  0.019462
 -370.131374   0.011657  0.019477
 -370.098063   0.011656  0.019509
 -370.064719   0.011674  0.019500
 -370.031408   0.011659  0.019485
 ...                ...       ...
  2762.641050  0.005847  0.009813
  2762.674394  0.005803  0.009814
  2762.707705  0.005834  0.009803
  2762.741048  0.005855  0.009810
  2762.774392  0.005837  0.009800
 
 [93994 rows x 2 columns],
 'Isosbestic':                     SI       PPT
 times                           
 -370.148046   0.011773  0.019487
 -370.114734   0.010041  0.017752
 -370.081391   0.011781  0.019512
 -370.048048   0.011775  0.019507
 -370.014736   0.011785  0.019465
 ...                ...       ...
  2762.657722  0.012615  0.020670
  2762.691033  0.012602  0.020660
  2762.724377  0.012575  0.020662
  2762.757720  0.012590  0.020651
  2762.791032  0.012588  0.020645
 
 [93994 rows x 2 columns]}